In [3]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [2]:
import os
import gc
import sys
srcpath = os.path.realpath('./../SourceFiles')
sys.path.append(srcpath)
#from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from pulse_sequence import EPR_pulse

In [ ]:
# Connect to the instrument

# Please choose appropriate address:
inst_addr = 'TCPIP::169.254.18.140::5025::SOCKET'

print('Connecting ..')
inst = pyte.open_session(inst_addr)
resp = inst.query("*IDN?")
print('Connected to: ' + resp)

# The paranoia-level is either 
# low (0), normal (1) or debug (2)
paranoia_level = 2

print ('Reset instrument ..')

# Reset the instrument
pyte.send_cmd(inst, '*CLS; *RST', paranoia_level)

print('Done')

In [ ]:
# Define the pulse sequence
srcpath = os.path.realpath('../../SourceFiles')
sys.path.append(srcpath)
from pulse_sequence import EPR_pulse
p = EPR_pulse(sampling_rate=2.5e9,Vpp=1)  

steps = [
    ('Square', 1e-9, {'amplitude': 1 ,'phase': 'x','f_c':0}),
    ('delay', 10e-6),
    ('Square', 10e-6, {'amplitude': 0.5 ,'phase': 'x','f_c':0}),
    ('delay', 10e-6),
    ('gaussian', 20e-6, {'amplitude': 0.5, 'phase': '-x','f_c':.51e6}),
    ('delay', 10e-6),
]

t_seq, y_seq = p.pulse_sequence(steps)
dig_y = p.DAC_output(y_seq) 


In [ ]:
# Download waveform to the AWT
# Download segments
ichan = 0
channb = ichan + 1
segnum = ichan % 2 + 1
wav = dig_y
seglen = len(wav)
print('Download wave to segment {0} of channel {1}'.format(segnum, channb))

# Select channel
cmd = ':INST:CHAN {0}'.format(channb)
inst.send_scpi_cmd(cmd)

# Define segment
cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
inst.send_scpi_cmd(cmd)

# Select the segment
cmd = ':TRAC:SEL {0}'.format(segnum)
inst.send_scpi_cmd(cmd)
    
# Send the binary-data:
inst.write_binary_data(':TRAC:DATA', wav)
    
resp = inst.send_scpi_query(':SYST:ERR?')
resp = resp.rstrip()
if not resp.startswith('0'):
    print('ERROR: "{0}" after writing binary values'.format(resp))

# Play the specified segment at the selected channel:
cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum)
inst.send_scpi_cmd(cmd)

# Turn on the output of the selected channel:
inst.send_scpi_cmd(':OUTP ON')

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

print()

resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)

del dig_y
del wav